In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# ----------------------------
# Load and Clean NGL Data
# ----------------------------


def load_ngl_data(filepath):
    # Load all sheets dynamically, starting from the correct header row
    xl = pd.ExcelFile(filepath)
    sheets = xl.sheet_names
    print(f"Available Sheets: {sheets}")
    
    ngl_data = {}
    for sheet in sheets:
        df = xl.parse(sheet, header=4)  # Start from row 5 (header=4)
        df = df.rename(columns={df.columns[0]: 'Date'})  # Rename the date column
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Convert date column
        df = df.dropna(subset=['Date'])  # Drop rows where date is NaT
        
        # Keep only the first two contracts (NG1 and NG2) for now
        df = df[['Date', df.columns[1], df.columns[2]]]
        df.columns = ['Date', 'NG1', 'NG2']
        
        ngl_data[sheet] = df
    
    return ngl_data

# File path
ngl_filepath = '/Users/markbogorad/Desktop/NYU_MFE/Semester_2/1.2_Trading_Energy_Derivatives_MATH_GA_2800/Oil_Project/Systematic_Energy_Trading/Data/NGLs.xlsx'

# Load the data
ngl_datasets = load_ngl_data(ngl_filepath)

# Display one of the datasets to verify
for key, df in ngl_datasets.items():
    print(f"\nFirst few rows of sheet: {key}")
    print(df.head())
    break  # Display only the first sheet for now



Available Sheets: ['NG', 'CAP', 'BAP', 'DAE', 'IBD', 'MMB', 'PCW', 'PGP']

First few rows of sheet: NG
        Date    NG1    NG2
0 2011-07-07  4.133  4.138
1 2011-07-08  4.205  4.204
2 2011-07-11  4.288  4.277
3 2011-07-12  4.333  4.312
4 2011-07-13  4.403  4.387


In [ ]:
def momentum_strategy(df, n=20):
    df["momentum_signal"] = np.where(df["F1"] > df["F1"].rolling(n).mean(), 1, 
                                     np.where(df["F1"] < df["F1"].rolling(n).mean(), -1, 0))

    df["momentum_pnl"] = df["momentum_signal"].shift(1) * df["dF"] * VOLUME
    df["momentum_equity"] = df["momentum_pnl"].cumsum()
    
    return df


In [ ]:
def compute_metrics(df, pnl_column):
    annual_pnl = df[pnl_column].sum() / (len(df) / 252)  # Annualized P&L
    sharpe_ratio = annual_pnl / (df[pnl_column].std() * np.sqrt(252))
    
    hwm = df[pnl_column].cummax()
    max_drawdown = (hwm - df[pnl_column]).max()
    return_on_drawdown = annual_pnl / max_drawdown if max_drawdown != 0 else np.nan
    
    return {
        "Annual PnL": annual_pnl,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "Return on Drawdown": return_on_drawdown
    }
